In [ ]:
%pip install -q ipylab

# Panels and Widgets

In [1]:
import anyio
import ipywidgets as ipw

import ipylab

app = await ipylab.App().ready()

## Panel

A `Panel` widget is the same as a `ipywidget.Box`, but with a `Title` that is used when the panel is added to the application shell.

In [2]:
panel = ipylab.Panel(children=[ipw.Dropdown()])

To quickly add the panel to the JupyterLab *shell* main area:

In [7]:
sc = await panel.add_to_shell(area='right', activate=False)

In [4]:
sc in panel.connections

True

In [5]:
sc in app.shell.connections

True

`sc` is a `ShellConnection`. 
It provides an `activate` method, and in further may have other features added.

In [9]:
await sc.activate()  # Will activate before returning to this notebook

Closing the connection will remove the panel from the shell but leave the panel open.

In [8]:
sc.close()

We can put it back in the shell.

In [9]:
sc = await panel.add_to_shell(mode=ipylab.InsertMode.split_right, activate=False)

In [10]:
# closable is on the widget in the shell rather than the panel, but we can set it using set_property.
await sc.set_property("title.closable", False)

The title label can be updated as required.

In [11]:
panel.title.label = "This panel has a dropdown"

We can close the panel and the view will disappear.

In [12]:
panel.close()

When the panel is closed sc is also closed.

In [13]:
sc

< CLOSED: ShellConnection(cid='ipylab-ShellConnection|ad24f5e5-0666-4671-8a94-cc5aa1681555') >

In the case of sliders and other widgets that fit on a single line, they can even be added to the top area directly:

In [14]:
slider = ipw.IntSlider()
await app.shell.add(slider, area=ipylab.Area.top)

ShellConnection(cid='ipylab-ShellConnection|f04c4893-b585-49dd-b046-a378d1439b54')

We can also remove it from the top area when we are done.

In [15]:
slider.close()

## SplitPanel
A split panel is a subclass of Panel that provides a draggable border between widgets, whose orientatation can be either horizontal or vertical.
Let's create a `SplitPanel` with a few widgets inside.

In [16]:
split_panel = ipylab.SplitPanel()
progress = ipw.IntProgress(
    value=7,
    min=0,
    max=100,
    step=1,
    description="Loading:",
    bar_style="info",
    orientation="horizontal",
    layout={"height": "30px"},
)
slider_ctrl = ipw.IntSlider(
    min=0,
    max=100,
    step=1,
    description="Slider Control:",
)

# link the slider to the progress bar
ipw.jslink((slider_ctrl, "value"), (progress, "value"))

# add the widgets to the split panel
split_panel.children = [progress, slider_ctrl]
ipw.Box(children=[split_panel], layout={"height": "100px"})

Box(children=(SplitPanel(children=(IntProgress(value=7, bar_style='info', description='Loading:', layout=Layou…

In [17]:
split_panel.title.label = "A SplitPanel "
split_panel.title.icon_class = "jp-PythonIcon"

> As an alternative to `icon_class`, a `Panel` can also use custom [icons](./icons.ipynb).

In [18]:
await split_panel.add_to_shell(area=ipylab.Area.main, mode=ipylab.InsertMode.split_bottom)

ShellConnection(cid='ipylab-ShellConnection|8c3fe00e-fb19-4cb2-b486-b3e336491068')

The orientation can be updated on the fly:

In [19]:
split_panel.orientation = "horizontal"

Let's put it back to `vertical`

In [20]:
split_panel.orientation = "vertical"

Just like with boxes, we can add an existing widget (the progress bar) more than once:

In [21]:
split_panel.children += (progress,)

Or add a new widget:

In [22]:
play = ipw.Play(min=0, max=100, step=1, description="Press play")
ipw.jslink((play, "value"), (slider_ctrl, "value"))
split_panel.children += (play,)

## Left and Right Areas

The same `SplitPanel` widget (or `Panel` or `Widget`) can be moved to the left area:

In [23]:
await split_panel.add_to_shell(area=ipylab.Area.left, rank=1000)
await split_panel.connections[0].activate()

Or to the right area:

In [24]:
await split_panel.add_to_shell(area=ipylab.Area.right, rank=1000)
await split_panel.connections[0].activate()

In [25]:
await app.shell.collapse_right()

Notice how it moved the widget instead of adding a second one?

This is the default behaviour.

To have multiple widgets, provide it with a new `cid` when 'adding' it to the shell.

In [26]:
await split_panel.add_to_shell(cid=ipylab.ShellConnection.to_cid(), mode=ipylab.InsertMode.split_right)

ShellConnection(cid='ipylab-ShellConnection|b5d24de4-34ff-4d16-96e7-f043a03e3d7f')

In [27]:
await split_panel.connections[0].activate()
split_panel.connections

(ShellConnection(cid='ipylab-ShellConnection|8c3fe00e-fb19-4cb2-b486-b3e336491068'),
 ShellConnection(cid='ipylab-ShellConnection|b5d24de4-34ff-4d16-96e7-f043a03e3d7f'))

In [28]:
split_panel.close()
await anyio.sleep(0.1)

In [29]:
split_panel.connections

()